# MA_DL : Codierung 64bit

In [1]:
!pip install facenet-pytorch # due to issues with the google cloud service

In [2]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models
from PIL import Image

from facenet_pytorch import MTCNN, InceptionResnetV1

dtype = torch.cuda.FloatTensor

## Model

In [3]:
class MultiLabel(nn.Sequential):
    def __init__(self, input_dim=512, output_dim=64):
        super(MultiLabel, self).__init__()
        self.l1 = nn.Linear(input_dim, input_dim)
        self.l2 = nn.Linear(input_dim, input_dim)
        self.l5 = nn.Linear(input_dim, output_dim)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.dropout(F.relu(self.l1(x)))
        x = self.dropout(F.relu(self.l2(x)))
        x = self.l5(x)
        return torch.sigmoid(x)

In [4]:
model = MultiLabel()
# model.load_state_dict(torch.load('dl64b_v4_new_414.pt'))
model.cuda()

MultiLabel(
  (l1): Linear(in_features=512, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l5): Linear(in_features=512, out_features=64, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

## Data

In [5]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()
to_image = transforms.ToPILImage()
transform = transforms.Compose([
    scaler, 
    to_tensor,
])

In [6]:
train_ds = datasets.ImageFolder('../../../data/small_data_new/train/', transform=transform)
train_ds.idx_to_class = {i:c for c, i in train_ds.class_to_idx.items()}
train_dl = DataLoader(train_ds, batch_size=1, num_workers=4)

valid_ds = datasets.ImageFolder('../../../data/small_data_new/valid/', transform=transform)
valid_ds.idx_to_class = {i:c for c, i in valid_ds.class_to_idx.items()}
valid_dl = DataLoader(valid_ds, batch_size=1, num_workers=4)

## Gesichtserkennung

In [7]:
mtcnn = MTCNN(
    image_size=160,
    thresholds=[0.6, 0.7, 0.7],
#        factor=0.709,
#        prewhiten=True,
    keep_all=True,
    device=torch.device('cuda')
    )

## Embeddings 

In [8]:
# Alternativ resnet
face_resnet = InceptionResnetV1(pretrained='casia-webface').eval()

## Training

In [9]:
def calculate_mistakes(pred, target):
    #pred = pred[0]
#    print('calculating mistakes : ')
#    print('   pred: ' + str(pred.shape))
#    print('   target: ' + str(target.shape))
    
    mistakes = 0
    
    if len(pred) != len(target):
        raise Exception('sizes of both tensors must match')
        
    for x,y in zip(pred, target):
        if round(x.item()) != y.item():
            mistakes = mistakes + 1
        
    return mistakes

In [10]:
loss_func = nn.BCELoss() #MultiLabelSoftMarginLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-5, momentum=0.9) #weight_decay=1e-4 

In [11]:
import matplotlib.pyplot as plt

In [13]:
epochs = 1000
for e in range(epochs):
    print("======================")
    print("Epoch : " + str(e))
    epoch_loss = 0
    epoch_mistakes = 0
    train_size = len(train_dl)
    
    # activate train mode
    model.train()

    for index, (data, target) in enumerate(train_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.cuda.FloatTensor([int(x) for x in train_ds.idx_to_class[target[0].item()]])

        emb = face_resnet(faces)    
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)
        
        pred = model.forward(data_v)
        optimizer.zero_grad()
        
        # calculate loss
        loss = loss_func(pred, target_v.float())
        epoch_loss = epoch_loss + loss
        # back prop
        loss.backward()
        optimizer.step()
        
        epoch_mistakes = epoch_mistakes + calculate_mistakes(pred, target_t)
        
    print("Train loss : " + str(epoch_loss.item()))
    print("Average errors : " + str(epoch_mistakes/train_size))
    
    # activate eval mode
    model.eval()
    
    valid_loss = 0
    valid_mistakes = 0
    valid_size = len(valid_dl)
    for index, (data, target) in enumerate(valid_dl):
        faces, prob = mtcnn(to_image(data[0]), return_prob=True)
        target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
        emb = face_resnet(faces) 
        data_v   = Variable(emb[0], requires_grad=False).type(dtype)
        target_v = Variable(target_t, requires_grad=False).type(dtype)

        pred = model.forward(data_v)
        loss = loss_func(pred, target_v.float())
        valid_loss = valid_loss + loss
        valid_mistakes = valid_mistakes + calculate_mistakes(pred, target_t)
        
        
    print("Valid loss : " + str(valid_loss.item()))
    print("Average valid errors : " + str(valid_mistakes/valid_size))

Epoch : 0
Train loss : 224.19540405273438
Average errors : 30.39814814814815
Valid loss : 28.956815719604492
Average valid errors : 28.80952380952381
Epoch : 1
Train loss : 223.15171813964844
Average errors : 28.811728395061728
Valid loss : 28.855300903320312
Average valid errors : 28.714285714285715
Epoch : 2
Train loss : 222.38619995117188
Average errors : 28.49074074074074
Valid loss : 28.78070831298828
Average valid errors : 28.642857142857142
Epoch : 3
Train loss : 221.80186462402344
Average errors : 28.61111111111111
Valid loss : 28.728723526000977
Average valid errors : 28.428571428571427
Epoch : 4
Train loss : 221.38229370117188
Average errors : 28.429012345679013
Valid loss : 28.695764541625977
Average valid errors : 28.595238095238095
Epoch : 5
Train loss : 221.14744567871094
Average errors : 28.432098765432098
Valid loss : 28.676109313964844
Average valid errors : 28.571428571428573
Epoch : 6
Train loss : 221.08656311035156
Average errors : 28.441358024691358
Valid loss : 28

Train loss : 194.80697631835938
Average errors : 22.46604938271605
Valid loss : 26.33649444580078
Average valid errors : 23.476190476190474
Epoch : 48
Train loss : 194.61984252929688
Average errors : 22.549382716049383
Valid loss : 26.293123245239258
Average valid errors : 23.428571428571427
Epoch : 49
Train loss : 192.8998565673828
Average errors : 22.333333333333332
Valid loss : 26.20618438720703
Average valid errors : 23.142857142857142
Epoch : 50
Train loss : 193.05413818359375
Average errors : 22.311728395061728
Valid loss : 26.158693313598633
Average valid errors : 23.357142857142858
Epoch : 51
Train loss : 191.89329528808594
Average errors : 22.117283950617285
Valid loss : 26.046199798583984
Average valid errors : 23.357142857142858
Epoch : 52
Train loss : 191.1691131591797
Average errors : 21.848765432098766
Valid loss : 26.0037841796875
Average valid errors : 22.952380952380953
Epoch : 53
Train loss : 189.8040313720703
Average errors : 21.651234567901234
Valid loss : 25.918216

Train loss : 149.02809143066406
Average errors : 15.614197530864198
Valid loss : 21.620771408081055
Average valid errors : 16.595238095238095
Epoch : 95
Train loss : 147.30467224121094
Average errors : 15.410493827160494
Valid loss : 21.588804244995117
Average valid errors : 16.761904761904763
Epoch : 96
Train loss : 146.14276123046875
Average errors : 15.29320987654321
Valid loss : 21.488021850585938
Average valid errors : 16.595238095238095
Epoch : 97
Train loss : 145.5131378173828
Average errors : 15.154320987654321
Valid loss : 21.30303382873535
Average valid errors : 16.333333333333332
Epoch : 98
Train loss : 144.85958862304688
Average errors : 15.13888888888889
Valid loss : 21.19042205810547
Average valid errors : 16.30952380952381
Epoch : 99
Train loss : 142.51272583007812
Average errors : 14.737654320987655
Valid loss : 21.144229888916016
Average valid errors : 16.214285714285715
Epoch : 100
Train loss : 143.23773193359375
Average errors : 15.006172839506172
Valid loss : 21.015

Valid loss : 16.931562423706055
Average valid errors : 11.30952380952381
Epoch : 142
Train loss : 114.01353454589844
Average errors : 10.808641975308642
Valid loss : 16.879968643188477
Average valid errors : 11.142857142857142
Epoch : 143
Train loss : 113.44871520996094
Average errors : 10.876543209876543
Valid loss : 16.752368927001953
Average valid errors : 11.238095238095237
Epoch : 144
Train loss : 110.6826400756836
Average errors : 10.753086419753087
Valid loss : 16.61093521118164
Average valid errors : 11.119047619047619
Epoch : 145
Train loss : 111.0166244506836
Average errors : 10.762345679012345
Valid loss : 16.451759338378906
Average valid errors : 10.904761904761905
Epoch : 146
Train loss : 109.5850830078125
Average errors : 10.552469135802468
Valid loss : 16.379858016967773
Average valid errors : 10.738095238095237
Epoch : 147
Train loss : 110.92385864257812
Average errors : 10.478395061728396
Valid loss : 16.23284149169922
Average valid errors : 10.857142857142858
Epoch : 

Train loss : 84.86157989501953
Average errors : 7.6419753086419755
Valid loss : 12.494583129882812
Average valid errors : 7.095238095238095
Epoch : 193
Train loss : 83.50479125976562
Average errors : 7.280864197530864
Valid loss : 12.462873458862305
Average valid errors : 7.166666666666667
Epoch : 194
Train loss : 84.85873413085938
Average errors : 7.669753086419753
Valid loss : 12.34877872467041
Average valid errors : 7.119047619047619
Epoch : 195
Train loss : 85.71028137207031
Average errors : 7.657407407407407
Valid loss : 12.250892639160156
Average valid errors : 7.119047619047619
Epoch : 196
Train loss : 82.73587036132812
Average errors : 7.280864197530864
Valid loss : 12.209741592407227
Average valid errors : 7.023809523809524
Epoch : 197
Train loss : 83.96636962890625
Average errors : 7.401234567901234
Valid loss : 12.176076889038086
Average valid errors : 7.095238095238095
Epoch : 198
Train loss : 82.42866516113281
Average errors : 7.410493827160494
Valid loss : 12.108935356140

...

Train loss : 48.95429992675781
Average errors : 3.691358024691358
Valid loss : 8.001142501831055
Average valid errors : 4.238095238095238
Epoch : 289
Train loss : 49.298988342285156
Average errors : 3.7160493827160495
Valid loss : 7.911017417907715
Average valid errors : 4.119047619047619
Epoch : 290
Train loss : 47.96339416503906
Average errors : 3.611111111111111
Valid loss : 7.841907024383545
Average valid errors : 4.333333333333333
Epoch : 291
Train loss : 47.521514892578125
Average errors : 3.462962962962963
Valid loss : 7.977667808532715
Average valid errors : 4.285714285714286
Epoch : 292
Train loss : 49.04172897338867
Average errors : 3.861111111111111
Valid loss : 7.878629207611084
Average valid errors : 4.023809523809524
Epoch : 293
Train loss : 48.371055603027344
Average errors : 3.675925925925926
Valid loss : 7.831089019775391
Average valid errors : 4.261904761904762
Epoch : 294
Train loss : 47.83097839355469
Average errors : 3.70679012345679
Valid loss : 7.885796546936035


Valid loss : 6.91947603225708
Average valid errors : 3.5714285714285716
Epoch : 336
Train loss : 35.76451110839844
Average errors : 2.509259259259259
Valid loss : 6.847597122192383
Average valid errors : 3.5952380952380953
Epoch : 337
Train loss : 36.7120475769043
Average errors : 2.5
Valid loss : 6.828153133392334
Average valid errors : 3.5476190476190474
Epoch : 338
Train loss : 36.37500762939453
Average errors : 2.4753086419753085
Valid loss : 6.841272830963135
Average valid errors : 3.6666666666666665
Epoch : 339
Train loss : 36.40493392944336
Average errors : 2.5123456790123457
Valid loss : 6.980077743530273
Average valid errors : 3.6904761904761907
Epoch : 340
Train loss : 37.733001708984375
Average errors : 2.567901234567901
Valid loss : 6.965533256530762
Average valid errors : 3.738095238095238
Epoch : 341
Train loss : 37.38720703125
Average errors : 2.6018518518518516
Valid loss : 6.785430908203125
Average valid errors : 3.5238095238095237
Epoch : 342
Train loss : 36.409900665

Train loss : 28.608970642089844
Average errors : 1.9475308641975309
Valid loss : 6.17864990234375
Average valid errors : 3.3095238095238093
Epoch : 384
Train loss : 29.988462448120117
Average errors : 1.9228395061728396
Valid loss : 6.150290012359619
Average valid errors : 3.4047619047619047
Epoch : 385
Train loss : 28.9362735748291
Average errors : 1.8364197530864197
Valid loss : 6.168980121612549
Average valid errors : 3.4285714285714284
Epoch : 386
Train loss : 28.789247512817383
Average errors : 1.876543209876543
Valid loss : 6.096630096435547
Average valid errors : 3.0952380952380953
Epoch : 387
Train loss : 28.477861404418945
Average errors : 1.876543209876543
Valid loss : 6.077791213989258
Average valid errors : 3.119047619047619
Epoch : 388
Train loss : 28.86834716796875
Average errors : 1.8981481481481481
Valid loss : 6.158958911895752
Average valid errors : 3.238095238095238
Epoch : 389
Train loss : 30.18092918395996
Average errors : 2.006172839506173
Valid loss : 6.160645484

Train loss : 23.066808700561523
Average errors : 1.4444444444444444
Valid loss : 5.870357513427734
Average valid errors : 2.9761904761904763
Epoch : 431
Train loss : 22.982669830322266
Average errors : 1.4969135802469136
Valid loss : 5.849902153015137
Average valid errors : 2.880952380952381
Epoch : 432
Train loss : 23.630464553833008
Average errors : 1.5401234567901234
Valid loss : 5.815445423126221
Average valid errors : 2.857142857142857
Epoch : 433
Train loss : 25.032255172729492
Average errors : 1.595679012345679
Valid loss : 5.682198524475098
Average valid errors : 2.7857142857142856
Epoch : 434
Train loss : 23.866971969604492
Average errors : 1.4876543209876543
Valid loss : 5.494746208190918
Average valid errors : 2.5714285714285716
Epoch : 435
Train loss : 22.9681339263916
Average errors : 1.4228395061728396
Valid loss : 5.641477584838867
Average valid errors : 2.738095238095238
Epoch : 436
Train loss : 23.642181396484375
Average errors : 1.3796296296296295
Valid loss : 5.73199

KeyboardInterrupt: 

In [14]:
torch.save(model.state_dict(), 'dl64b_v5_new_445.pt')

## Testing 

In [62]:
def tensor_to_binary(tensor):
    if tensor.dim() != 2:
        raise ValueException('tensor must have two dimensions')

    for i, x in enumerate(tensor):
        for j, y in enumerate(x) :
            if y.item() < 0.5:
                tensor[i][j] = 0
            elif y.item() >= 0.5:
                tensor[i][j] = 1

    return tensor

In [65]:
for i, t in train_ds:
    print(i.shape)
    faces, prob = mtcnn(to_image(i), return_prob=True)
    target_t = torch.FloatTensor([int(x) for x in valid_ds.idx_to_class[target[0].item()]])
        
    emb = face_resnet(faces) 
    data_v   = Variable(emb[0], requires_grad=False).type(dtype)
    target_v = Variable(target_t, requires_grad=False).type(dtype)

    pred = model.forward(data_v)
    print(pred.shape)
    print(tensor_to_binary(pred.unsqueeze(0)))

torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
         1., 0., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 1.

torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 1., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
         0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
         0., 1., 0., 0., 0., 0., 1., 0., 

torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
         1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1.,
         1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         0., 1., 0., 0., 1., 0., 1., 1., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 1., 1., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 

torch.Size([64])
tensor([[0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.,
         0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0.,
         0., 1., 0., 1., 1., 0., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 1., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
         0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 1., 

torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1.,
         1., 0., 0., 1., 0., 1., 0., 1., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
         1., 1., 0., 1., 0., 1., 0., 0., 1., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
         1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 1., 1., 1., 

...

torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
         0., 1., 0., 0., 1., 0., 1., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1.,
         1., 1., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         1., 1., 0., 0., 1., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1.,
         0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
         0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
         1., 1., 1., 0., 1., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0.,
         0., 0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1.,
         1., 0., 0., 1., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 1.,
         0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
         1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
         1., 1., 1., 0., 0., 0., 0., 0., 

torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0.,
         0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 0., 0., 0.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0.,
         0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1.,
         0., 0., 0., 1., 0., 1., 1., 1., 

torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1.,
         0., 1., 1., 0., 0., 0., 0., 0., 1., 1.]], device='cuda:0',
       grad_fn=<AsStridedBackward>)
torch.Size([3, 224, 224])
torch.Size([64])
tensor([[1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1.,
         0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
         1., 0., 0., 1., 0., 0., 1., 0., 

In [47]:
!ls ../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011

Tom_Hanks_0001.jpg  Tom_Hanks_0004.jpg	Tom_Hanks_0007.jpg  Tom_Hanks_0010.jpg
Tom_Hanks_0002.jpg  Tom_Hanks_0005.jpg	Tom_Hanks_0008.jpg
Tom_Hanks_0003.jpg  Tom_Hanks_0006.jpg	Tom_Hanks_0009.jpg


In [ ]:
path = '../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/'
for i in os.listdir(path):
        Image.open(path + i)
Image.open('../../data/small_data/train/1111100010001101110100100001010010101001001010001110110110000011/T')

In [ ]:
model.forward()